# Imports

In [4]:
import os
import pickle
from datetime import datetime
from typing import Tuple, Dict, Any, List

import numpy as np
import pandas as pd
from scipy import sparse
from scipy.sparse import csr_matrix, lil_matrix, save_npz, load_npz
from sklearn.metrics.pairwise import cosine_similarity
from math import sqrt
from IPython.display import display, Markdown
from pathlib import Path

try:
    from tqdm import tqdm
except Exception:
    tqdm = lambda x, **k: x

print("Imports complete.")

Imports complete.


# Path Definitions (Portable)

In [5]:
# ได้ Path ของโฟลเดอร์โปรเจกต์ (ที่ไฟล์ .ipynb นี้อยู่)
PROJECT_ROOT = Path(os.getcwd())

# สร้าง Path ไปยังโฟลเดอร์ย่อยต่างๆ
PROCESSED_PATH = PROJECT_ROOT / "processed"
CLEANED_PATH = PROCESSED_PATH / "cleaned"
PREPROCESS_PATH = PROCESSED_PATH / "preprocess"
MODEL_PATH = PROCESSED_PATH / "models"

# Utility function สำหรับ Log
def log(msg: str, level: str = "INFO") -> None:
    ts = datetime.now().strftime("%Y-m-d %H:%M:%S")
    print(f"[{level}] {ts} | {msg}")

log(f"Project Root (This Notebook's location): {PROJECT_ROOT}")
log(f"Model Path set to: {MODEL_PATH}")
log(f"Cleaned Data Path set to: {CLEANED_PATH}")

[INFO] 2025-m-d 07:40:18 | Project Root (This Notebook's location): C:\Users\nonth\Documents\Movie_Predict
[INFO] 2025-m-d 07:40:18 | Model Path set to: C:\Users\nonth\Documents\Movie_Predict\processed\models
[INFO] 2025-m-d 07:40:18 | Cleaned Data Path set to: C:\Users\nonth\Documents\Movie_Predict\processed\cleaned


# Helper Functions

In [18]:
# --- ฟังก์ชันสำหรับโหลด SVD Artifacts ---
def load_svd_artifacts(model_dir: Path) -> Dict[str, Any]:
    log("Loading SVD artifacts from disk...")
    U = np.load(model_dir / "svd_U.npy")
    Sigma = np.load(model_dir / "svd_Sigma.npy")
    Vt = np.load(model_dir / "svd_Vt.npy")
    user_mean = np.load(model_dir / "svd_user_mean.npy")
    
    with open(model_dir / "svd_user_index.pkl", "rb") as f:
        user_index = pickle.load(f)
    with open(model_dir / "svd_movie_index.pkl", "rb") as f:
        movie_index = pickle.load(f)
    with open(model_dir / "svd_reverse_user_index.pkl", "rb") as f:
        reverse_user_index = pickle.load(f)
    with open(model_dir / "svd_reverse_movie_index.pkl", "rb") as f:
        reverse_movie_index = pickle.load(f)
        
    log("Loaded SVD artifacts")
    return {
        "U": U, "Sigma": Sigma, "Vt": Vt, "user_mean": user_mean,
        "user_index": user_index, "movie_index": movie_index,
        "reverse_user_index": reverse_user_index, "reverse_movie_index": reverse_movie_index
    }

# --- ฟังก์ชันสำหรับ Test 1: Content-Based ---
def get_content_based_recs(movie_title: str, top_n: int = 10) -> pd.DataFrame:
    log(f"Finding Content-Based recommendations for: '{movie_title}'")
    
    # 1. ค้นหา movieId จาก title
    movie_row = movies_global[movies_global['title'].str.contains(movie_title, case=False, na=False)]
    if movie_row.empty:
        log(f"Movie not found: {movie_title}", "WARN")
        return pd.DataFrame()
    
    movie_id = movie_row.iloc[0]['movieId']
    log(f"Found movieId: {movie_id}")

    # 2. ค้นหา index ใน sparse matrix
    idx_arr = np.where(movie_ids_global == movie_id)[0]
    if idx_arr.size == 0:
        log(f"MovieId {movie_id} not found in TF-IDF matrix (no content data).", "WARN")
        return pd.DataFrame()
    
    idx = int(idx_arr[0])

    # 3. ดึง Top-N ที่คล้ายกันจาก sim_sparse
    row_indices = sim_sparse.rows[idx]
    row_data = sim_sparse.data[idx]
    
    if len(row_data) == 0:
        log(f"No similar movies found for index {idx}.", "WARN")
        return pd.DataFrame()

    # 4. Map กลับเป็น movieId และ title
    similar_movie_ids = [movie_ids_global[i] for i in row_indices]
    
    result = movies_global[movies_global.movieId.isin(similar_movie_ids)][['movieId', 'title']].copy()
    score_map = dict(zip(similar_movie_ids, row_data))
    result['similarity_score'] = result['movieId'].map(score_map)
    
    # ไม่เอาตัวเอง และจัดอันดับ
    result = result[result.movieId != movie_id].sort_values('similarity_score', ascending=False)
    
    return result.head(top_n)

# --- ฟังก์ชันสำหรับ Test 2.1: CF (SVD) (Input: User) ---
def get_cf_recs_for_user(user_id: int, top_n: int = 10) -> pd.DataFrame:
    log(f"Finding CF (SVD) recommendations for User {user_id}")
    
    if user_id not in svd_user_index:
        log(f"User ID {user_id} not found in SVD training set.", "WARN")
        return pd.DataFrame()
        
    u_idx = svd_user_index[user_id]
    
    # ทำนายคะแนน (user_vector @ Vt) + user_mean
    user_vector = np.dot(U[u_idx, :], Sigma)  
    preds = np.dot(user_vector, Vt) + svd_user_mean[u_idx]  
    
    # กรองหนังที่เคยดูแล้ว
    seen_movie_ids = set(ratings_global[ratings_global.userId == user_id]['movieId'])
    
    # Map index กลับไปเป็น movieId
    recs = []
    for i in range(len(preds)):
        if i in svd_reverse_movie_index: # ตรวจสอบว่า movie index นี้มีใน mapping
            movie_id = svd_reverse_movie_index[i]
            if movie_id not in seen_movie_ids:
                recs.append((movie_id, preds[i]))

    # จัดอันดับ
    recs.sort(key=lambda x: x[1], reverse=True)
    
    # Join ชื่อหนัง
    top_movie_ids = [mid for mid, score in recs[:top_n]]
    result = movies_global[movies_global.movieId.isin(top_movie_ids)][['movieId', 'title']].copy()
    score_map = dict(recs[:top_n])
    result['predicted_rating'] = result['movieId'].map(score_map)
    
    return result.sort_values('predicted_rating', ascending=False)

# --- ฟังก์ชันสำหรับ Test 2.2: CF (SVD) (Input: Movie) ---
def get_cf_recs_for_movie(movie_title: str, top_n: int = 10) -> pd.DataFrame:
    log(f"Finding best Users for Movie '{movie_title}' using CF (SVD)")
    
    # 1. ค้นหา movieId จาก title
    movie_row = movies_global[movies_global['title'].str.contains(movie_title, case=False, na=False)]
    if movie_row.empty:
        log(f"Movie not found: {movie_title}", "WARN")
        return pd.DataFrame()
    
    movie_id = movie_row.iloc[0]['movieId']
    
    if movie_id not in svd_movie_index:
        log(f"MovieId {movie_id} not in SVD training set.", "WARN")
        return pd.DataFrame()
        
    m_idx = svd_movie_index[movie_id]

    # 2. ทำนายคะแนน (U @ movie_vector) + user_mean
    movie_factors = Vt[:, m_idx]                 # ได้ vector คุณลักษณะแฝงของหนัง (Shape k,)
    movie_vector = np.dot(Sigma, movie_factors)  # นำไปถ่วงน้ำหนักด้วย Sigma (Shape k,)
    preds_all_users = np.dot(U, movie_vector) + svd_user_mean # (n_users,)
    
    # 3. Map กลับเป็น userId
    recs = []
    for i in range(len(preds_all_users)):
        if i in svd_reverse_user_index:
            user_id = svd_reverse_user_index[i]
            recs.append((user_id, preds_all_users[i]))
            
    # 4. จัดอันดับ
    recs.sort(key=lambda x: x[1], reverse=True)
    
    # 5. สร้าง DataFrame
    top_users = recs[:top_n]
    result = pd.DataFrame(top_users, columns=['userId', 'predicted_rating'])
    
    return result

# --- ฟังก์ชันสำหรับ Test 3: Hybrid (เหมือนเดิม) ---
# (หมายเหตุ: 2 ฟังก์ชันนี้ถูกคัดลอกมาจาก Cell 3 ของคำตอบก่อนหน้า)
def hybrid_score(userId: int, movieId: int, alpha: float = 0.7, top_k: int = 50) -> float:
    try:
        svd_row = svd_preds_df_global[svd_preds_df_global.movieId == movieId]
        svd_score = float(svd_row.pred_rating.values[0]) if not svd_row.empty else np.nan
    except Exception:
        svd_score = np.nan

    if sim_sparse is None or len(movie_ids_global) == 0:
        content_score = np.nan
    else:
        idx_arr = np.where(movie_ids_global == movieId)[0]
        if idx_arr.size == 0:
            content_score = np.nan
        else:
            idx = int(idx_arr[0])
            row = sim_sparse.rows[idx]
            data = sim_sparse.data[idx]
            if len(data) == 0:
                content_score = np.nan
            else:
                content_score = float(np.nanmean(data[:top_k]))

    if np.isnan(svd_score) and np.isnan(content_score): return np.nan
    if np.isnan(svd_score): return content_score
    if np.isnan(content_score): return svd_score
    return alpha * svd_score + (1.0 - alpha) * content_score

def recommend_movies(userId: int, top_n: int = 10, alpha: float = 0.7, top_k_content: int = 50) -> pd.DataFrame:
    global svd_preds_df_global 
    try:
        # คำนวณ SVD preds 'ทั้งหมด' สำหรับ user นี้ 'ครั้งเดียว'
        svd_preds_df_global = get_cf_recs_for_user(userId, top_n=len(svd_movie_index))
        # เปลี่ยนชื่อคอลัมน์ให้ตรงกับที่ hybrid_score คาดหวัง
        svd_preds_df_global = svd_preds_df_global.rename(columns={'predicted_rating': 'pred_rating'})
    except ValueError:
        return pd.DataFrame(columns=['movieId', 'title', 'hybrid_score', 'reason'])

    seen = set(ratings_global.loc[ratings_global.userId == userId, 'movieId'].unique())
    candidates = [mid for mid in movie_ids_global if mid not in seen]
    
    log(f"Scoring {len(candidates)} candidate movies for user {userId} (Hybrid)...")
    
    scores = []
    for mid in tqdm(candidates, desc=f"Scoring user {userId} (Hybrid)"):
        score = hybrid_score(userId, mid, alpha=alpha, top_k=top_k_content)
        if not np.isnan(score):
            scores.append((mid, score))
            
    if len(scores) == 0:
        log(f"No candidate scores for user {userId}", "WARN")
        return pd.DataFrame(columns=['movieId', 'title', 'hybrid_score'])

    scores.sort(key=lambda x: x[1], reverse=True)
    top_scores = scores[:top_n]
    top_movie_ids = [mid for mid, s in top_scores]
    
    result = movies_global[movies_global.movieId.isin(top_movie_ids)][['movieId', 'title']].copy()
    score_map = dict(top_scores)
    result['hybrid_score'] = result['movieId'].map(score_map)
    
    return result.sort_values('hybrid_score', ascending=False).reset_index(drop=True)

log("All helper functions defined.")

[INFO] 2025-m-d 07:54:17 | All helper functions defined.


# LOAD DATA & MODELS

In [7]:
log("--- STARTING ONE-TIME LOAD ---")

# 1. โหลด DataFrames (เก็บเป็น Global)
log("Loading DataFrames (movies, ratings)...")
movies_global = pd.read_csv(CLEANED_PATH / "movies_cleaned_f.csv")
ratings_global = pd.read_csv(CLEANED_PATH / "ratings_cleaned_f.csv")

# 2. โหลด Movie IDs (สำหรับ Content-Based)
log("Loading TF-IDF reduced data (for movie_ids)...")
tfidf_reduced_df = pd.read_csv(PREPROCESS_PATH / "movies_tfidf_reduced.csv")
movie_ids_global = tfidf_reduced_df["movieId"].to_numpy()

# 3. โหลด Content Similarity Matrix (Global)
log("Loading Content Similarity matrix...")
sim_sparse = load_npz(MODEL_PATH / "content_similarity_sparse.npz").tolil() # .tolil() เพื่อให้เข้าถึง row ได้เร็ว

# 4. โหลด SVD Artifacts (Global)
artifacts = load_svd_artifacts(MODEL_PATH)
U = artifacts["U"]
Sigma = artifacts["Sigma"]
Vt = artifacts["Vt"]
svd_user_mean = artifacts["user_mean"]
svd_user_index = artifacts["user_index"]
svd_movie_index = artifacts["movie_index"]
svd_reverse_user_index = artifacts["reverse_user_index"]
svd_reverse_movie_index = artifacts["reverse_movie_index"]

# 5. สร้างตัวแปร Global ว่างๆ สำหรับเก็บ SVD preds ของ user
svd_preds_df_global = pd.DataFrame() 

log("--- ALL ARTIFACTS LOADED AND READY ---")

[INFO] 2025-m-d 07:41:22 | --- STARTING ONE-TIME LOAD ---
[INFO] 2025-m-d 07:41:22 | Loading DataFrames (movies, ratings)...
[INFO] 2025-m-d 07:42:11 | Loading TF-IDF reduced data (for movie_ids)...
[INFO] 2025-m-d 07:42:13 | Loading Content Similarity matrix...
[INFO] 2025-m-d 07:42:13 | Loading SVD artifacts from disk...
[INFO] 2025-m-d 07:42:15 | Loaded SVD artifacts
[INFO] 2025-m-d 07:42:15 | --- ALL ARTIFACTS LOADED AND READY ---


# TEST 1: Content-Based (Input: Movie)

In [16]:
# --- 📍 INPUT ---
MOVIE_TITLE_TO_TEST = "Jumanji"  # <--- เปลี่ยนชื่อหนังตรงนี้
TOP_N_SIMILAR = 10
# ---------------

display(Markdown(f"### 1. Content-Based Recommendations (Similar to '{MOVIE_TITLE_TO_TEST}')"))
cb_recs = get_content_based_recs(MOVIE_TITLE_TO_TEST, top_n=TOP_N_SIMILAR)
display(cb_recs)

### 1. Content-Based Recommendations (Similar to 'Jumanji')

[INFO] 2025-m-d 07:51:03 | Finding Content-Based recommendations for: 'Jumanji'
[INFO] 2025-m-d 07:51:03 | Found movieId: 2


,movieId,title,similarity_score
4933,5038,"Flight of Dragons, The",0.915349
21615,111362,X-Men: Days of Future Past,0.904710
25102,122914,Avengers: Infinity War - Part II,0.889960
85885,286901,The Flash,0.888910
9268,27618,"Sound of Thunder, A",0.887921
3702,3802,Freejack,0.883562
85887,286905,Indiana Jones and the Dial of Destiny,0.882369
80668,270696,The Adam Project,0.879870
11514,52287,Meet the Robinsons,0.879759
2600,2692,Run Lola Run (Lola rennt),0.876279


# TEST 2.1: Collaborative Filtering (SVD) (Input: User)

In [14]:
# --- 📍 INPUT ---
USER_ID_TO_TEST = 42  # <--- เปลี่ยน userId ตรงนี้
TOP_N_FOR_USER = 10
# ---------------

display(Markdown(f"### 2.1 CF (SVD) Recommendations (For User {USER_ID_TO_TEST})"))
cf_user_recs = get_cf_recs_for_user(USER_ID_TO_TEST, top_n=TOP_N_FOR_USER)
display(cf_user_recs)

### 2.1 CF (SVD) Recommendations (For User 42)

[INFO] 2025-m-d 07:47:34 | Finding CF (SVD) recommendations for User 42


,movieId,title,predicted_rating
536,541,Blade Runner,5.925704
734,750,Dr. Strangelove or: How I Learned to Stop Worr...,5.742744
1183,1214,Alien,5.716575
1108,1136,Monty Python and the Holy Grail,5.648053
1070,1097,E.T. the Extra-Terrestrial,5.527467
1167,1197,"Princess Bride, The",5.516415
1170,1200,Aliens,5.433599
1194,1225,Amadeus,5.344448
1175,1206,"Clockwork Orange, A",5.297365
1351,1387,Jaws,5.237160


# TEST 2.2: Collaborative Filtering (SVD) (Input: Movie)

In [19]:
# --- 📍 INPUT ---
MOVIE_TITLE_FOR_USERS = "Forrest Gump"  # <--- เปลี่ยนชื่อหนังตรงนี้
TOP_N_USERS = 10
# -----------------

display(Markdown(f"### 2.2 CF (SVD) Recommendations (Users for '{MOVIE_TITLE_FOR_USERS}')"))
cf_movie_recs = get_cf_recs_for_movie(MOVIE_TITLE_FOR_USERS, top_n=TOP_N_USERS)
display(cf_movie_recs)

### 2.2 CF (SVD) Recommendations (Users for 'Forrest Gump')

[INFO] 2025-m-d 07:54:21 | Finding best Users for Movie 'Forrest Gump' using CF (SVD)


,userId,predicted_rating
0,114901,12.118600
1,194456,11.711371
2,104941,11.622508
3,114185,11.491132
4,79649,11.412620
5,153690,11.391299
6,149158,11.370983
7,172322,11.360653
8,200110,11.354931
9,189014,11.340290


# TEST 3: Hybrid Model (Input: User)

In [13]:
# --- 📍 INPUT ---
USER_ID_HYBRID_TEST = 42  # <--- เปลี่ยน userId ตรงนี้
TOP_N_HYBRID = 10
# ---------------

display(Markdown(f"### 3. Hybrid Recommendations (For User {USER_ID_HYBRID_TEST})"))
hybrid_recs = recommend_movies(userId=USER_ID_HYBRID_TEST, top_n=TOP_N_HYBRID)
display(hybrid_recs)

### 3. Hybrid Recommendations (For User 42)

[INFO] 2025-m-d 07:44:05 | Finding CF (SVD) recommendations for User 42
[INFO] 2025-m-d 07:44:06 | Scoring 87537 candidate movies for user 42 (Hybrid)...


Scoring user 42 (Hybrid): 100%|████████████████████████████████████████████████| 87537/87537 [00:48<00:00, 1817.00it/s]


,movieId,title,hybrid_score
0,541,Blade Runner,4.378204
1,750,Dr. Strangelove or: How I Learned to Stop Worr...,4.275218
2,1214,Alien,4.273540
3,1136,Monty Python and the Holy Grail,4.239331
4,1097,E.T. the Extra-Terrestrial,4.126972
5,1197,"Princess Bride, The",4.093783
6,1200,Aliens,4.080624
7,1225,Amadeus,3.994549
8,1206,"Clockwork Orange, A",3.925565
9,1387,Jaws,3.922256
